<a href="https://colab.research.google.com/github/GHROTH-L/crawler/blob/main/%E7%88%AC%E5%8F%96COVID19%E5%96%AE%E6%97%A5%E7%A2%BA%E8%A8%BA%E4%BA%BA%E6%95%B8%E8%88%87%E6%AD%BB%E4%BA%A1%E6%95%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#使用手冊注意


1.   發病日與死亡日的日期數請對照一樣
2.   要更改發病死亡日請從兩個DEF去做更改
3.old 與 new差別在2023/3/20起，配合中央流行疫情指揮中心修訂「嚴重特殊傳染性肺炎」病例定義，該疾病名稱修訂為「嚴重特殊傳染性肺炎(併發症)」，新修訂病例統計資料請於「嚴重特殊傳染性肺炎(併發症)」查詢




In [200]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
import pandas as pd
from google.colab import files

In [201]:
#設置查詢年月日與cookies(發病日的)
def old_sick(url1):
  for_data={
    '__RequestVerificationToken': '_gzm0KabA-okYOJq00k92hQy47mD9vOpMrnQMzqq6VkvdT8l4WHpYuX4OCq4cjVtAw-gu3F5-7jeFxVaJREiknNrEMkLNrXqlfnb9dwRFeo1',
    'pty_Q': 'N',
    'pty_disease': '19CoV',
    'pty_period_value': 'ymd', #年月日
    'minYear': '2020',
    'pty_y_s': '2020',
    'pty_m_s': '01',
    'pty_d_s': '01',
    'pty_y_e': '2023',
    'pty_m_e': '05',
    'pty_d_e': '31',
    'pty_date_type_value': '3',
    'pty_immigration_value': ['0,1'], #移入與本土都有
    'pty_sickclass_value': 'determined_cnt',
    'pty_area_text': '全國',
    'pty_city_text': '全部',
    'pty_town_text': '全部'

    }
  cookies={
    '__RequestVerificationToken':'phD-ED1aUic2mOrCMtAb5WvEHWuAcOdzbplnhPPELpVaIKnWYigCgthqSahMspq8hCAmgZlgk6b8WjrwZEoFN6774jDGogyYOHp71cdqP9c1',
    '_gat_gtag_UA_45325352_1':'1'
  }
  r1 = requests.post(url1, data= for_data, headers = headers, cookies=cookies)
  soup = BeautifulSoup(r1.text,"html.parser")
  k = soup.find("div", class_="detailMain blameChart").script
  script = str(k)
  return script

In [202]:
#設置查詢年月日與cookies死亡日)
def old_death(url2):
  for_data2={
    '__RequestVerificationToken': '3G_AXFanPFbbHbNrONq0yn-jMMerEuMnB_3xkuSOIdWc87GkJCVJMUQ3evCqyxNeXdS0dKamL10_osVhrraAt8ZGv7yKWtiOrEO4_Z3nYVY1',
    'pty_Q': 'N',
    'pty_disease': '19CoV',
    'pty_period_value': 'ymd',
    'minYear': '2020',
    'pty_y_s': '2020',
    'pty_m_s': '01',
    'pty_d_s': '01',
    'pty_y_e': '2023',
    'pty_m_e': '05',
    'pty_d_e': '31',
    'pty_date_type_value': '10',
    'pty_immigration_value': ['0,1'],
    'pty_sickclass_value': 'determined_cnt',
    'pty_area_text': '全國',
    'pty_city_text': '全部',
    'pty_town_text': '全部'

  }
  cookies2={
      '__RequestVerificationToken':'phD-ED1aUic2mOrCMtAb5WvEHWuAcOdzbplnhPPELpVaIKnWYigCgthqSahMspq8hCAmgZlgk6b8WjrwZEoFN6774jDGogyYOHp71cdqP9c1'
  }
  r2 = requests.post(url2, data= for_data2, headers = headers, cookies=cookies2)
  soup2 = BeautifulSoup(r2.text,"html.parser")
  k2 = soup2.find("div", class_="detailMain blameChart").script
  script2 = str(k2)
  return script2

In [203]:
#處理確診發病日的部分 以及死亡日
def old_scraping(script,script2):
  pattern_categories = r'"xAxis_categories":\[(.*?)\]' #正規化搜尋
  pattern_data = r'"data":\[(.*?)\]'
  '''
  前缀表示一个原始字符串（raw string）。使用原始字符串可以防止Python解释器对字符串中的特殊字符进行转义处理。
  正则表达式中经常包含许多反斜杠（\），用于表示特殊字符或元字符。如果不使用原始字符串，那么需要对反斜杠进行转义，例如使用\\表示一个反斜杠字符。但是，使用原始字符串时，不需要进行额外的转义，可以直接书写正则表达式模式。
  '''
  #發病日
  matches_categories = re.search(pattern_categories, script)
  matches_data = re.search(pattern_data, script)

  if matches_categories and matches_data:
    categories_str = matches_categories.group(1)
    data_str = matches_data.group(1)
    
    xAxis_categories = [category.strip('"') for category in categories_str.split(",")]
    data = [category.strip('"') for category in data_str.split(",")]
  #死亡日
  matches_categories2 = re.search(pattern_categories, script2)
  matches_data2 = re.search(pattern_data, script2)    
  if matches_categories2 and matches_data2:
    categories_str2 = matches_categories2.group(1)
    data_str2 = matches_data2.group(1)
    
    xAxis_categories2 = [category.strip('"') for category in categories_str2.split(",")]
    data2 = [category.strip('"') for category in data_str2.split(",")]
    #很奇怪爬取的資料 死亡數會少20200229這一天 不過這一天是0
    xAxis_categories2.insert(59,'20200229')
    data2.insert(59,'0')
    #print("xAxis_categories:", xAxis_categories)
    #print("data:", data)
    #print("data:", dat2)
  return [xAxis_categories, data, data2]

In [204]:
#創造字典 轉 輸出csv
def all_old_data(xAxis_categories,data,data2):
  all_old_data ={
      "日期":xAxis_categories,
      "確診人數(發病日)":data,
      "死亡人數(死亡日)":data2

  }
  df= pd.DataFrame(all_old_data)

  # 保存DataFrame为CSV文件
  df.to_csv('all_old_data.csv', index=False)

  # 下载CSV文件
  files.download('all_old_data.csv')

In [205]:
#設置查詢年月日與cookies(發病日的)
def new_sick(url3):
  for_data={
    '__RequestVerificationToken': 'xWV8VwKfk6v8Wag6BeqZDa9FAZD7DH-ZQD2ttWgCee-bOA89p6pyttzTgIUWvlGjYEWE6L6ihF07-uar3OLG_QGr_92eiNuuuBrBxLlZwuQ1',
    'pty_Q': 'N',
    'pty_disease': '19CVS',
    'pty_period_value': 'ymd', #年月日
    'minYear': '2023',
    'pty_y_s': '2023',
    'pty_m_s': '01',
    'pty_d_s': '01',
    'pty_y_e': '2023',
    'pty_m_e': '05',
    'pty_d_e': '31',
    'pty_date_type_value': '3',
    'pty_immigration_value': ['0,1'], #移入與本土都有
    'pty_sickclass_value': 'determined_cnt',
    'pty_area_text': '全國',
    'pty_city_text': '全部',
    'pty_town_text': '全部'

    }
  cookies3={
    '__RequestVerificationToken':'phD-ED1aUic2mOrCMtAb5WvEHWuAcOdzbplnhPPELpVaIKnWYigCgthqSahMspq8hCAmgZlgk6b8WjrwZEoFN6774jDGogyYOHp71cdqP9c1',
    '_ga_KB9TE8J28D':'GS1.1.1685524938.4.1.1685524957.0.0.0',
    'Guide_Disease':'true'
  }
  r3 = requests.post(url3, data= for_data, headers = headers, cookies=cookies3)
  soup3 = BeautifulSoup(r3.text,"html.parser")
  k3 = soup3.find("div", class_="detailMain blameChart").script
  script3 = str(k3)
  return script3

In [206]:
#處理確診發病日
def new_scraping(script3):
  pattern_categories = r'"xAxis_categories":\[(.*?)\]' #正規化搜尋
  pattern_data = r'"data":\[(.*?)\]'
  #發病日
  matches_categories3 = re.search(pattern_categories, script3)
  matches_data3 = re.search(pattern_data, script3)

  if matches_categories3 and matches_data3:
    categories_str3 = matches_categories3.group(1)
    data_str3 = matches_data3.group(1)
    
    xAxis_categories3 = [category.strip('"') for category in categories_str3.split(",")]
    data3 = [category.strip('"') for category in data_str3.split(",")]
  return [xAxis_categories3, data3]

In [207]:
#創造字典 轉 輸出csv
def all_new_data(xAxis_categories3,data3):
  all_new_data ={
      "日期":xAxis_categories3,
      "確診人數(發病日)":data3
  }
  df= pd.DataFrame(all_new_data)

  # 保存DataFrame为CSV文件
  df.to_csv('all_new_data.csv', index=False)

  # 下载CSV文件
  files.download('all_new_data.csv')

In [208]:
if __name__ == "__main__":
  url1 = 'https://nidss.cdc.gov.tw/nndss/disease?id=19CoV' #舊的發病日
  url2 = 'https://nidss.cdc.gov.tw/nndss/deadmap?id=19CoV&type=3' #舊的死亡日
  url3 = 'https://nidss.cdc.gov.tw/nndss/disease?id=19CVS' #新的發病日
  headers = {"user-agen" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
  print("爬取確診資料")
  script = old_sick(url1)
  time.sleep(10)
  print("爬取死亡資料")
  script2 = old_death(url2)
  time.sleep(10)
  print("爬取新定義的資料")
  script3 = new_sick(url3)
  old_scrap = old_scraping(script,script2)
  new_scrap = new_scraping(script3)
  all_old_data(old_scrap[0],old_scrap[1],old_scrap[2])
  all_new_data(new_scrap[0],new_scrap[1])
  


爬取確診資料
爬取死亡資料
爬取新定義的資料


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>